<a href="https://colab.research.google.com/github/Zmtbiker12/Colaboratory/blob/main/HW2B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from plotly.subplots import make_subplots
from scipy.integrate import quad

In [9]:
R = 8.314
Tc = 647.096 #Kelvin
Pc = 22.064e6 #Mega Pa
w = 0.348 # acentric factor
omega = 0.08664
psi = 0.42748
b = omega*R*Tc/Pc

In [10]:
def calc_abes(T):
  Tr = T/Tc # Reduced tempurature
  alpha = (1+(0.48+1.574*w-0.176*w**2)*(1-Tr**(0.5)))**2
  e=0.
  s=1.
  a = psi*alpha*(R**2)*(Tc**2)/(Pc)
  
  return a,b,e,s

  #This function calculates a, b, epsilon and sigma based upon the SRK parameters and a given tempurature in kelvins

In [11]:
def srkP(v, T):
  a,b,e,s = calc_abes(T)
  return R*T/(v-b) - a/((v+e*b)*(v+s*b))

  #Takes a volume and tempurature valuen and outputs the pressure using SRK EOS

In [12]:
v = np.logspace(np.log10(1.01*b), -1, 1000)

In [13]:
fig = make_subplots()
for T in np.arange(275, 900, 25):
  fig.add_scatter(x=v, y=srkP(v,T), mode='lines', name = F'{T}K')
fig.update_xaxes(type='log')
fig.update_yaxes(range=[1e-7, 1e8])
fig.update_layout(width =800, height = 600)

In [14]:
def cubic_roots(a,b,c):
  Q = ((a**2)-(3*b))/9
  R = ((2*a**3)-(9*a*b)+27*c)/54
  if R**2<Q**3:
    theta = np.arccos(R/Q**(1.5))
    return -2*np.sqrt(Q)*np.cos(np.array([theta/3, (theta-2*np.pi)/3, (theta+2*np.pi)/3 ])) - (a/3)
  else:
    A = -(R+(np.sqrt(R**2 - Q**3)))**(1/3)
    B = 0 if A == 0 else (Q/A)
    return (A+B) - a/3

#Root solver

In [15]:
def cubic_v(P,T):
  a,b,e,s = calc_abes(T)
  beta = b*P/R/T
  q = a/(b*R*T)
  zs = cubic_roots(((e+s)*beta - beta - 1), (e*s*beta**2 - (e+s)*beta**2 - (e+s)*beta + q*beta),(-e*s*beta**3 - e*s*beta**2 - q*beta**2))

  return zs*R*T/P
#Returns volume roots

In [17]:
def maxwell_a(P,T):
  vl, vm, vg = cubic_v(P,T)
  return quad(lambda v: P - srkP(v,T), vl, vm)[0], quad(lambda v: srkP(v,T)- P, vm, vg)[0]

In [18]:
maxwell_a(91e3,373)

(9154.889447297845, 9143.0263427625)